In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

# =========================
# 1. Carregar dados
# =========================
file_path = 'data/btc_limpo.csv'
df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')

prices = df['Close'].values.reshape(-1,1)

scaler = MinMaxScaler(feature_range=(0,1))
prices_scaled = scaler.fit_transform(prices)

# =========================
# 2. Função para criar sequências
# =========================
def create_sequences(data, seq_len=60):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

SEQ_LEN = 60
X, y = create_sequences(prices_scaled, SEQ_LEN)
X = np.expand_dims(X, axis=-1)

# dividir treino/val/teste (70/20/10)
split1 = int(0.7*len(X))
split2 = int(0.9*len(X))

X_train, y_train = X[:split1], y[:split1]
X_val, y_val = X[split1:split2], y[split1:split2]
X_test, y_test = X[split2:], y[split2:]

# =========================
# 3. Funções de modelos
# =========================
def build_lstm(units=50, dropout=0.2):
    model = Sequential([
        LSTM(units, return_sequences=False, input_shape=(SEQ_LEN,1)),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_gru(units=50, dropout=0.2):
    model = Sequential([
        GRU(units, return_sequences=False, input_shape=(SEQ_LEN,1)),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# =========================
# 4. Grid de Experimentos
# =========================
experiments = []

param_grid = [
    {'type': 'LSTM', 'units': 50, 'dropout': 0.2, 'batch_size': 32},
    {'type': 'LSTM', 'units': 100, 'dropout': 0.3, 'batch_size': 64},
    {'type': 'GRU', 'units': 50, 'dropout': 0.2, 'batch_size': 32},
    {'type': 'GRU', 'units': 100, 'dropout': 0.3, 'batch_size': 64},
]

results_dir = Path('results')
outputs_dir = Path('outputs')
results_dir.mkdir(exist_ok=True)
outputs_dir.mkdir(exist_ok=True)

for params in param_grid:
    model_name = f"{params['type']}_u{params['units']}_d{params['dropout']}_b{params['batch_size']}"
    print(f"\n=== Rodando experimento: {model_name} ===")

    if params['type'] == 'LSTM':
        model = build_lstm(params['units'], params['dropout'])
    else:
        model = build_gru(params['units'], params['dropout'])

    ckpt_path = results_dir / f"{model_name}.keras"

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(filepath=str(ckpt_path), monitor='val_loss', save_best_only=True)
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=params['batch_size'],
        callbacks=callbacks,
        verbose=1
    )

    # carregar melhor modelo
    model = load_model(str(ckpt_path))

    # avaliação no teste
    y_pred_scaled = model.predict(X_test, verbose=0).flatten()
    y_pred = scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).flatten()
    y_true = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()

    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    experiments.append({
        'model': model_name,
        'MAPE': mape,
        'MAE': mae,
        'RMSE': rmse
    })

    # salvar gráfico
    plt.figure(figsize=(10,5))
    plt.plot(y_true, label='Real')
    plt.plot(y_pred, label='Previsto')
    plt.title(f"{model_name} - Previsão no Teste")
    plt.legend()
    plt.savefig(outputs_dir / f"{model_name}_forecast.png")
    plt.close()

# =========================
# 5. Resultados finais
# =========================
results_df = pd.DataFrame(experiments)
results_df = results_df.sort_values(by='MAPE')

print("\n📊 Ranking final dos modelos:")
print(results_df)

results_df.to_csv(results_dir / 'comparacao_0610.csv', index=False)



=== Rodando experimento: LSTM_u50_d0.2_b32 ===
Epoch 1/30


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 0.0131 - val_loss: 0.0104
Epoch 2/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0020 - val_loss: 0.0013
Epoch 3/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 4/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0013 - val_loss: 0.0016
Epoch 5/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0013 - val_loss: 0.0020
Epoch 6/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0011 - val_loss: 9.9266e-04
Epoch 7/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 8/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0010 - val_loss: 0.0016
Epoch 9/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0010 - val_loss: 9.1914e-04
Epoch 10/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 9.5002e-04 - val_loss: 8.5539e-04
Epoch 11/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 8.4719e-04 - val_loss: 0.0023
Epoch 12/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - 

C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 91ms/step - loss: 0.0107 - val_loss: 0.0154
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0020 - val_loss: 0.0045
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 0.0015 - val_loss: 0.0024
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - loss: 0.0011 - val_loss: 0.0014
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 9.9002e-04 - val_loss: 0.0017
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 9.8537e-04 - val_loss: 0.0010
Epoch 9/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0010 - val_loss: 0.0013
Epoch 10/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 8.9767e-04 - val_loss: 9.3139e-04
Epoch 11/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 9.2467e-04 - val_loss: 9.9192e-04
Epoch 12/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/ste

C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - loss: 0.0142 - val_loss: 0.0200
Epoch 2/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0020 - val_loss: 7.1421e-04
Epoch 3/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0011 - val_loss: 7.0593e-04
Epoch 4/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 9.0761e-04 - val_loss: 7.7797e-04
Epoch 5/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 9.1885e-04 - val_loss: 8.1429e-04
Epoch 6/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 9.0844e-04 - val_loss: 5.4989e-04
Epoch 7/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 7.7455e-04 - val_loss: 5.9552e-04
Epoch 8/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 7.3881e-04 - val_loss: 0.0012
Epoch 9/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 7.8898e-04 - val_loss: 9.0848e-04
Epoch 10/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 7.3113e-04 - val_loss: 7.3179e-04
Epoch 11/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 6.6360e-04 - val_loss: 7.0015e-04

=== R

C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 94ms/step - loss: 0.0171 - val_loss: 0.0177
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0033 - val_loss: 0.0021
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0012 - val_loss: 7.7227e-04
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 9.1996e-04 - val_loss: 6.2548e-04
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 8.0429e-04 - val_loss: 7.3612e-04
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 8.3623e-04 - val_loss: 5.7639e-04
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 8.2772e-04 - val_loss: 7.4010e-04
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 7.1594e-04 - val_loss: 7.4122e-04
Epoch 9/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - loss: 7.5435e-04 - val_loss: 5.3968e-04
Epoch 10/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 7.0363e-04 - val_loss: 5.1722e-04
Epoch 11/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 6.9745e-04 - val_loss: 6.3666e-04
Epoch 